In [14]:
import pandas as pd
import sqlite3
import json
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
df_movies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")


In [3]:
df_credits = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

In [4]:
conn = sqlite3.connect('df.movies1.db')

In [5]:
# Escribir los DataFrames en tablas SQL
df_movies.to_sql('movies', conn, index=False, if_exists='replace')
df_credits.to_sql('credits', conn, index=False, if_exists='replace')

4803

In [6]:
# Commit y cerrar la conexión
conn.commit()
conn.close()

In [7]:
conn = sqlite3.connect('df.movies1.db')
# Realizar la operación de unión SQL
query = '''
    SELECT movies.*, credits.*
    FROM movies
    JOIN credits ON movies.title = credits.title
'''

# Leer el resultado en un nuevo DataFrame
df_completo = pd.read_sql_query(query, conn)

In [8]:
# Cerrar la conexión
conn.close()

# Mostrar el DataFrame resultante
df_completo.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [9]:
df_completo = df_completo[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]


In [10]:
df_completo = df_completo.loc[:, ~df_completo.columns.duplicated()]


In [11]:
df_completo.columns

Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew'], dtype='object')

In [12]:
def load_json_safe(json_str, default_value = None):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return default_value
    
df_completo["genres"] = df_completo["genres"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)
df_completo["keywords"] = df_completo["keywords"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)

df_completo["cast"] = df_completo["cast"].apply(lambda x: [item["name"] for item in json.loads(x)][:3] if pd.notna(x) else None)

df_completo["crew"] = df_completo["crew"].apply(lambda x: " ".join([crew_member['name'] for crew_member in load_json_safe(x) if crew_member['job'] == 'Director']))

df_completo["overview"] = df_completo["overview"].apply(lambda x: [x])

df_completo.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


In [13]:
df_completo["overview"] = df_completo["overview"].apply(lambda x: [str(x)])
df_completo["genres"] = df_completo["genres"].apply(lambda x: [str(genre) for genre in x])
df_completo["keywords"] = df_completo["keywords"].apply(lambda x: [str(keyword) for keyword in x])
df_completo["cast"] = df_completo["cast"].apply(lambda x: [str(actor) for actor in x])
df_completo["crew"] = df_completo["crew"].apply(lambda x: [str(crew_member) for crew_member in x])

df_completo["tags"] = df_completo["overview"] + df_completo["genres"] + df_completo["keywords"] + df_completo["cast"] + df_completo["crew"]
df_completo["tags"] = df_completo["tags"].apply(lambda x: ",".join(x).replace(",", " "))

df_completo.drop(columns = ["genres", "keywords", "cast", "crew", "overview"], inplace = True)

df_completo.iloc[0].tags

"['In the 22nd century  a paraplegic Marine is dispatched to the moon Pandora on a unique mission  but becomes torn between following orders and protecting an alien civilization.'] Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d Sam Worthington Zoe Saldana Sigourney Weaver J a m e s   C a m e r o n"

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

# Vectorizar los datos usando TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_completo["tags"])

# Calcular similitud de coseno entre todas las películas
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)*100

def get_movie_recommendations(movie_title, cosine_sim=cosine_sim):
    # Obtener el índice de la película de entrada
    movie_index = df_completo[df_completo["title"] == movie_title].index[0]

    # Obtener las puntuaciones de similitud de coseno para todas las películas
    sim_scores = list(enumerate(cosine_sim[movie_index]))

    # Ordenar las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las películas más similares (excluyendo la película de entrada)
    similar_movies = sim_scores[1:6] 

    # Obtener los nombres de las películas recomendadas
    recommended_movies = [(df_completo["title"][i], score) for i, score in similar_movies]

    return recommended_movies

# Ejemplo de uso

while True:
    titulo = input("Ingresa un título: ")
    if titulo in df_completo["title"].values:
        break
    else:
        print("¡Intenta con otro título!")


recommendations = get_movie_recommendations(titulo)

print("Recomendaciones para '{}':".format(titulo))
for movie, score in recommendations:
    print("- Película: {}, Porcentaje de parecidos: {:.2f}%".format(movie, score))


¡Intenta con otro título!
¡Intenta con otro título!
¡Intenta con otro título!
¡Intenta con otro título!
Recomendaciones para 'Avatar':
- Película: Aliens, Porcentaje de parecidos: 28.66%
- Película: Alien³, Porcentaje de parecidos: 27.23%
- Película: Mission to Mars, Porcentaje de parecidos: 25.92%
- Película: Moonraker, Porcentaje de parecidos: 25.23%
- Película: Alien, Porcentaje de parecidos: 24.99%
